In [30]:
def plot_pred(hof):
    func = toolbox.compile(expr=hof[0])
    y_hat = []
    y = []
    y_r = []
    for a, i in enumerate(t):
        y_hat.append(func(i))
        y.append(p[a])
        y_r.append(p_r[a])
        
    from matplotlib import pyplot as plt
    plt.plot(t,y)
    plt.plot(t, y_hat)
    plt.plot(t, y_r)
    plt.xlabel('time')  # x-axis
    plt.ylabel('x(t)')  # y-axis
    plt.title('Data')  # title
    plt.grid()  # grid
    plt.legend(['with error','predicted','real'])
    plt.show()  # plot show
###############################import###############################
import operator
import math
import random
import pandas as pd
import numpy
import time
import os
import dill
from sympy import *

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
from scoop import futures
from pandas import DataFrame

from concurrent.futures import ProcessPoolExecutor
import matplotlib.pyplot as plt
import multiprocessing as mp

###################################################################

###################### Define functions############################
def safeDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1
    
def sqrt(left):
    if left>=0:
        return left**0.5
    else:
        return 0
    
def expe(left):
    if left>=0:
        return numpy.exp(-left)
    else:
        return 0




pset = gp.PrimitiveSet("MAIN", 2)
pset.addPrimitive(operator.add, 2)
#pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(expe, 1)
#set.addPrimitive(safeDiv, 2)
pset.addPrimitive(sqrt,1)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
try:
    pset.addEphemeralConstant("rand", random.randint(1,5))
except:
    print("")
#pset.addEphemeralConstant("rand", lambda: random.randint(1,5))
#pset.addEphemeralConstant("rand"+str(ran1)+str(ran2), lambda: random.randint(min(ran1,ran2),max(ran1,ran2)))
pset.renameArguments(ARG0='x')
pset.renameArguments(ARG1='y')



###################################################################
pool = mp.Pool()

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
#toolbox.register("map", futures.map)

#########################input DATA#################################

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)

    # Evaluate the mean squared error between the expression
    sqerrors = ((func(x,y) - f[x,y])**2 for x,y in points)

    try:
        return max(sqerrors),
    except IndexError:
        return max(sqerrors),
    
toolbox.register("map", pool.map)


g = pd.read_csv('os.csv')
p = g['x']
g_r = pd.read_csv('os_r.csv')
p_r = g_r['x']

t = list(numpy.arange(0,10,0.01))

tt=t.append(t)

f = {}
for i in range(len(t)):
    f[t[i]] = p[i]
    

ff = {}
for i in range(len(t)):
    ff[t[i]] = p_r[i]
    
    
toolbox.register("evaluate", evalSymbReg, points=tt)
toolbox.register("select", tools.selTournament, tournsize=100)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=1, max_=3)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=6))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=6))


def main():
    pop = toolbox.population(n=5000)
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)
    
    
    #############################################C.O, Muta,Gener##########
    try:
        pop, log = algorithms.eaSimple(pop, toolbox, 0.8, 0.8, 5, stats=mstats,
                                       halloffame=hof, verbose=True)
        print (hof[0])
        plot_pred(hof)
        return hof
    except MemoryError:
        print (hof[0])
        plot_pred(hof)
        return hof

    except KeyboardInterrupt:
        print (hof[0])
        plot_pred(hof)
        return hof



if __name__ == "__main__":
    hof = main()
    

/home/ban94gy/ban/local/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/ban94gy/ban/local/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


KeyError: 150

In [1]:
import operator
import math
import random
import pandas as pd
import numpy
import time
import os
import dill

from sympy import *
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
from scoop import futures

from concurrent.futures import ProcessPoolExecutor
import matplotlib.pyplot as plt
import multiprocessing as mp

###################################################################

###################### Define functions############################
def sqrt(left):
    if left>=0:
        return left**0.5
    else:
        return 0

pset = gp.PrimitiveSet("MAIN", 2) 
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(sqrt,1)

pset.addEphemeralConstant("rand",lambda: random.randint(1,10)/10)

    
#pset.addEphemeralConstant("rand", lambda: random.randint(1,5))
#pset.addEphemeralConstant("rand"+str(ran1)+str(ran2), lambda: random.randint(min(ran1,ran2),max(ran1,ran2)))
pset.renameArguments(ARG0='f1')
pset.renameArguments(ARG1='f2')

In [4]:
from sympy import *
###################################################################

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
# toolbox.register("map", futures.map)

#########################input DATA#################################

def evalSymbReg(individual, points):
    
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    
    # Evaluate the mean squared error between the expression
    sqerrors = ((func(f1,f2) + f)**2 for f,f1,f2 in points)
    
    try:
        return max(sqerrors),
    except IndexError:
        return max(sqerrors),
    
    

g = pd.read_csv('os_f.csv')
p = g['x']
g1 = pd.read_csv('os_df.csv')
p1 = g1['x']
g2 = pd.read_csv('os_ddf.csv')
p2 = g2['x']
pool = mp.Pool()

t = list(numpy.arange(0,0.5,0.01))

f = {}
f1 = {}
f2 = {}


for i in range(len(t)):
    f[i] = p[i]
    f1[i] = p1[i]
    f2[i] = p2[i]


    
ptf1 = []
ptf2 = []
for i in range(len(t)):
    ptf1.append(f1[i])
    ptf2.append(f2[i])

pts = []
for i in range(len(t)):
    pts.append((f[i], f1[i], f2[i]))
    
toolbox.register("evaluate", evalSymbReg, points=pts)

toolbox.register("map", pool.map)

toolbox.register("select", tools.selTournament, tournsize=100)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=1, max_=3)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=4))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=4))



def main():
    
    pop = toolbox.population(n=10000)
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    
    #############################################C.O, Muta,Gener##########
    try:
        pop, log = algorithms.eaSimple(pop, toolbox, 0.8, 0.8, 500, stats=mstats,
                                       halloffame=hof, verbose=True)
        print (hof[0])


    except MemoryError:
        print (hof[0])


    except KeyboardInterrupt:                                                                                                        
        print (hof[0])



if __name__ == "__main__":
    main()

/home/ban94gy/ban/local/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/ban94gy/ban/local/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


   	      	  fitness   	    size   
   	      	------------	-----------
gen	nevals	max    	min 	max	min
0  	10000 	1.6e+19	0.36	15 	2  
1  	9588  	9.03808e+16	0.25	19 	2  
2  	9615  	3.6e+15    	0.01	20 	2  
3  	9583  	1e+16      	1.10934e-31	23 	2  
4  	9613  	5.87414e+20	1.10934e-31	22 	2  
5  	9585  	1.72463e+21	1.10934e-31	23 	2  


Process ForkPoolWorker-115:
Process ForkPoolWorker-111:
Process ForkPoolWorker-109:
Process ForkPoolWorker-112:
Process ForkPoolWorker-114:
Process ForkPoolWorker-103:
Process ForkPoolWorker-118:
Process ForkPoolWorker-119:
Process ForkPoolWorker-107:
Process ForkPoolWorker-106:
Process ForkPoolWorker-113:


mul(mul(0.1, 0.2), mul(f2, 0.5))


Traceback (most recent call last):
Process ForkPoolWorker-92:
Process ForkPoolWorker-85:
Process ForkPoolWorker-117:
Process ForkPoolWorker-102:
Process ForkPoolWorker-82:
Process ForkPoolWorker-116:
Process ForkPoolWorker-120:
Process ForkPoolWorker-86:
Process ForkPoolWorker-108:
Process ForkPoolWorker-84:
Process ForkPoolWorker-83:
Process ForkPoolWorker-81:
Process ForkPoolWorker-88:
Process ForkPoolWorker-100:
Process ForkPoolWorker-104:
Process ForkPoolWorker-90:
Traceback (most recent call last):
Process ForkPoolWorker-87:
Process ForkPoolWorker-96:
Process ForkPoolWorker-91:
Process ForkPoolWorker-99:
Process ForkPoolWorker-89:
Process ForkPoolWorker-94:
Process ForkPoolWorker-93:
Process ForkPoolWorker-97:
Process ForkPoolWorker-95:
Process ForkPoolWorker-105:
Process ForkPoolWorker-110:
Process ForkPoolWorker-98:
Process ForkPoolWorker-101:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last

  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", 

  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self.

SyntaxError: invalid syntax (<ipython-input-7-444025390096>, line 1)

In [97]:
toolbox.register

<bound method Toolbox.register of <deap.base.Toolbox object at 0x7fe7b6d0e828>>

In [99]:
deap.base.Toolbox

NameError: name 'deap' is not defined

In [47]:
import pandas as pd
import numpy 

g = pd.read_csv('os.csv')
p = g['x']
g1 = pd.read_csv('os_r.csv')
p1 = g1['x']
g2 = pd.read_csv('os_rr.csv')
p2 = g2['x']


t = list(numpy.arange(0,1.5,0.01))

f = {}
f1 = {}
f2 = {}

for i in range(len(t)):
    f[i] = p[i]
    f1[i] = p1[i]
    f2[i] = p2[i]

pts = []
for i in range(len(t)):
    pts.append((f[i], f1[i], f2[i]))
def F(a,b):
    F=b
    return F
    
# Evaluate the mean squared error between the expression
sqerrors=((F(f1,f2)+4*f)**2 for f, f1, f2 in pts)


In [48]:
max(sqerrors)

7.8886090522101181e-31

In [40]:
print(sqerrors)

SyntaxError: invalid syntax (<ipython-input-40-69fbf7ab094d>, line 1)

In [4]:
 base.Fitness

deap.base.Fitness

In [5]:
from sympy import *
x=Symbol('x')
f = Symbol('f')
f1 = Symbol('f1')
f2 = Symbol('f2')
const = Symbol('const')

def add(a,b):
    return a+b

def expe(a):
    return exp(-a)

def mul(a,b):
    return a*b

def sub(a,b):
    return a-b

def neg(a):
    return -a

def fun(a):
    y=print(a)
    return y

const = 1

k=mul(mul(0.1, 0.2), mul(f2, 0.5))









In [6]:
k

0.01*f2

In [7]:
evalSymbReg

<function __main__.evalSymbReg>

In [6]:
k={}
for i in range(len(pts)):
    k[i] = f[i] + f1[i] + f2[i]


IndexError: invalid index to scalar variable.

In [54]:
g = pd.read_csv('os.csv')
p = g['x']
g1 = pd.read_csv('os_r.csv')
p1 = g1['x']
g2 = pd.read_csv('os_rr.csv')
p2 = g2['x']


t = list(numpy.arange(0,5,0.01))

f = []
f1 = []
f2 = []

for i in range(len(t)):
    f.append(p[i])
    f1.append(p1[i])
    f2.append(p2[i])

pts = []
for i in range(len(t)):
    pts.append((f[i], f1[i], f2[i]))

table = {'f' : f , 'f2' : f2}

sqerrors = {}

for f,f2 in f,f2:
# Evaluate the mean squared error between the expression
    sqerrors = (f2 + 16*f)**2 
    
sqerr = []

for i in range(len(pts)):
# Evaluate the mean squared error between the expression
    sqerr.append(sqerrors[i])

ValueError: too many values to unpack (expected 2)

In [8]:
math.fsum(sqerr)/len(pts)

1.918352974874152e-30

In [55]:
f

[1.0,
 0.99920010666097803,
 0.99680170630261944,
 0.99280863585386636,
 0.98722728337562682,
 0.98006657784124163,
 0.97133797485202955,
 0.96105543831077078,
 0.94923541808244083,
 0.93589682367793481,
 0.92106099400288521,
 0.90475166321996336,
 0.88699492277928416,
 0.86781917967764988,
 0.84725511101341611,
 0.82533561490967833,
 0.80209575788429266,
 0.77757271875092793,
 0.75180572914089505,
 0.7248360107409052,
 0.69670670934716539,
 0.66746282584130812,
 0.63715114419858021,
 0.60582015664346278,
 0.57351998607245669,
 0.54030230586813977,
 0.50622025723277841,
 0.47132836417374002,
 0.4356824462767121,
 0.39933952940627321,
 0.36235775447667362,
 0.32479628443877623,
 0.28671520963195551,
 0.2481754516523729,
 0.20923866589141926,
 0.16996714290024079,
 0.13042370873814554,
 0.09067162446430968,
 0.050774484933579181,
 0.010796117058267392,
 -0.029199522301288812,
 -0.069148448654062181,
 -0.10898675223987113,
 -0.14865070027136365,
 -0.1880768388928801,
 -0.22720209469308711

In [34]:
sqerr

[0.0,
 0.0,
 3.1554436208840472e-30,
 1.2621774483536189e-29,
 0.0,
 3.1554436208840472e-30,
 3.1554436208840472e-30,
 1.2621774483536189e-29,
 0.0,
 3.1554436208840472e-30,
 3.1554436208840472e-30,
 0.0,
 0.0,
 0.0,
 3.1554436208840472e-30,
 3.1554436208840472e-30,
 1.2621774483536189e-29,
 3.1554436208840472e-30,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 3.1554436208840472e-30,
 0.0,
 0.0,
 0.0,
 7.8886090522101181e-31,
 0.0,
 0.0,
 1.9721522630525295e-31,
 7.8886090522101181e-31,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 4.9303806576313238e-32,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 7.8886090522101181e-31,
 3.1554436208840472e-30,
 7.8886090522101181e-31,
 0.0,
 0.0,
 0.0,
 0.0,
 3.1554436208840472e-30,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.2621774483536189e-29,
 3.1554436208840472e-30,
 3.1554436208840472e-30,
 3.1554436208840472e-30,
 0.0,
 3.1554436208840472e-30,
 3.1554436208840472e-30,
 0.0,
 3.1554436208840472e-30,
 0.0,
 0.0,
 3.1554436208840472e-30,
 3.155443620